In [7]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException,NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [17]:
def job():
    #set the full path for the download directory.
    download_directory = os.path.join(os.getcwd(),'NewsToday') #(getcwd->get current working directory)

    options = webdriver.ChromeOptions()

    prefs = {
        'download_default_directory': download_directory,
        'plugins.always_open_pdf_externally': True
    }
    options.add_experimental_option('prefs', prefs)

    # option.add_argument("--headless")

    driver = webdriver.Chrome(options = options)
    driver.get("https://epaper.gorkhapatraonline.com/single/gorkhapatra")

    try:
        #wait for the pdf link to be present.
        pdf_links = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href,'.pdf')]")))

        for pdf_link in pdf_links:
            try:
                #Open the link in new tab.
                ActionChains(driver).key_down(Keys.CONTROL).click(pdf_link).key_up(Keys.CONTROL).perform()

                # key_down---> key press
                # key_up-----> key release

                # Switch to the new opened pdf link.
                driver.switch_to.window(driver.window_handles[-1])   # -1 ---> recently opened.(As in List mah -1 --> last ko.)

                #wait until download button is visible.
                download_button = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "//button[@id='download' and @title = 'Save']")))

                # Use pyautogui to click on the Download button (ActionChain can also be used bur=t sometimes it maynot work so we use pyautogui)
                ActionChains(driver).move_to_element(download_button).click().perform()

                #Allow sometime for download to complete.
                time.sleep(15)

            except (TimeoutException, NoSuchElementException) as e:
                print(f"Error Processing PDF Link: {e}")


            finally:
                #close all current tab
                driver.close()

                #switch back to main tab
                driver.switch_to.window(driver.window_handles[0])

    finally:
        driver.quit()

In [ ]:
# Execute the function directory.
job()

In [ ]:
# Issues that can occur:

# Version mismatch between webdriver and Chrome.